# Neural Networks

Nelle prossime due ore (circa) proveremo a **esplorare alcuni aspetti pratici che riguardano le reti neurali**: (i) l'implementazione della struttura della rete e (ii) il meccanismo di addestramento. 
Per farlo, ci serviremo di un linguaggio di programmazione e di un editor di testo.

## Strumenti

Prima di tutto, usiamo, come editor di testo, un _Google Colaboratory Notebook_ (se non sai cos'è un notebook puoi [dare un occhio qui](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html)).
In breve, questo strumento ci permette di scrivere testo formattato e codice in un unico file e di eseguirlo in un ambiente online (pronto all'uso).

## Prerequisiti 

Il tutorial assume la conoscenza dei seguenti concetti basilari: 
- **operazioni tra tensori** (vettori multidimensionali) che sono solitamente oggetto di un corso base di algebra lineare, 
- qualche nozione di **teoria delle probabilità**,
- **un po' di Python** o di un linguaggio ad oggetti similare.

# La storia di Frank

![il piccolo franco](https://tripleampersand.org/wp-content/uploads/2020/04/1.png)

Come sicuramente il lettore saprà, il padre del _perceptron_, l'algoritmo alla base di tutte le reti neurali, è **Frank Rosenblatt**, vissuto tra il 1928 e il 1971.
Frank riprende il lavoro di due colleghi, Warren McCulloch e Walter Pitts, che intorno al 1943 studiano le analogie tra il meccanismo di attivazione delle cellule neuronali e l'output delle porte logiche.

> L'intuizione è che il segnale (input) si accumula nel neurone fino al raggiungimento di una soglia critica. Se questa soglia viene superata allora il neurone si attiva ed emette un segnale in uscita (output).

![neurone](https://sebastianraschka.com/images/blog/2015/singlelayer_neural_networks_files/perceptron_neuron.png)

Rosenblatt immagina un modo di apprendere quando un neurone si attiva, così da poter costrure un classificatore lineare binario, in grado di separare linearmente dei segnali tra due classi.

Un po' più formalmente, il percettrone corrisponde ad un modello per l'aggiornamento dei pesi $w$ associati ad un vettore di input $x$ (la rappresentazione del segnale). L'input ed i relativi pesi si "accumulano" nel percettrone, la quantità risultante viene processata da una **funzione di attivazione** che produce l'output (la classe della predizione).

In figura, il primo valore dell'input $x_0=1$ ed il suo peso $w_0$ prendono il nome di **bias**, un valore che regolarizza l'output dell'algoritmo ed evita problemi di _overfitting_.

![percettrone](https://sebastianraschka.com/images/blog/2015/singlelayer_neural_networks_files/perceptron_schematic.png)

Infine, la forma del percettrone così come è stato ideato da Rosenblatt:

$$
\hat y = f( \sum_{i=1} x_i w_i + \textit{bias})
$$

Si noti che $x_0*w_0$ è separato dalla sommatoria e chiamato $\textit{bias}$.

# L'algoritmo di Rosenblatt

L'algoritmo, nella sua semplicità, prevede i seguenti passaggi:
1. Inizializzo il vettore dei pesi $w$ con valori _opportuni_
2. Per ogni input $x$
  - Calcolo il valore dell'output $\hat y$ con la funzione di attivazione
  - Aggiorno il vettore dei pesi $w$

## La funzione di attivazione $f$

Concentriamoci sul calcolo del valore $\hat y$, prima di tutto abbiamo bisogno di definire la funzione di attivazione.

Nonostante la varietà di funzioni possibili, la prima funzione di attivazione pensata da Rosenblatt è molto semplice:
se il segno del valore di accumulo è positivo allora l'output è 1, -1 altrimenti.

In [16]:
def sign(n):
  if n>0:
    return 1
  else:
    return -1

In [17]:
def relu(n):
  if n<0:
    return 0
  else:
    return n

In [18]:
relu(10000)

10000

In [19]:
sign(1000000)

1

Possiamo adesso scrivere l'algoritmo _semplificato_ per il percettrone di Rosenblatt.

In [20]:
def perceptron(x, w, bias, f=sign):
  sigma = 0
  for i in range(len(x)):
    sigma += x[i] * w[i]
  sigma += bias 
  return f(sigma)

In [21]:
import numpy as np

def np_perceptron(x, w, bias, f=sign):
  return f(np.sum(np.dot(x, w)) + bias)

Testiamo il nostro codice con un semplice input formato da due variabili: il punto di coordinate 0 e 1.

In [22]:
# dichiaro l'input del percettrone
x = [0,1]

import random
# inizializzo una casualmente la lista di pesi 
w = [random.uniform(-1,1) for i in range(len(x))]
# inizializza casualmente il bias
bias = random.uniform(-1,1)

# assegno la funzione di attivazione (in questo caso ReLU)
f = relu

# calcolo il valore output del precettrone
y_pred = perceptron(x, w, bias, relu) # try np_perceptron

In [23]:
y_pred

0.12439262228192782

Il lettore avrà notato che attualmente non abbiamo una strategia di aggiornamento dei pesi, il nostro modello infatti non sta veramente apprendendo, sta soltanto calcolando l'output.

Per farlo dobbiamo **calcolare l'errore commesso dal classificatore** e aggiornare i pesi tenendo in considerazione la differenza (se presente) tra la classe reale e la predizione. 

## L'addestramento (o aggiornamento dei pesi)

L'aggiornamento del vettore dei pesi $w$ deve essere fatto per ogni peso $w_i$, in generale vogliamo modificare la quantità $w_i$ di un certo $\Delta y_i$.

$$
w_i = w_i + \Delta y_i
$$

Il $\Delta y_i$ rappresenta la differenza tra il valore reale (*ground truth*) e la predizione, in altre parole l'**errore** del percettrone.

$$
\Delta y = \hat{y} - y
$$

Nel processo di addestramento introduciamo una nuova variabile: il **learning rate** (abbreviato con `lr`), un fattore di moltiplicazione "piccolo" a piacere, che permettere di *regolare* l'aggiornamento dei pesi.

Il suo ruolo è quello di ridurre il valore con cui aggiorniamo i pesi della rete, per non "esagerare".

In [24]:
def train_perceptron(x, w, bias, y, lr=0.1, f=sign):
  # calcoliamo la predizione
  y_pred = perceptron(x, w, bias, f)
  # calcoliamo la differenza tra la predizione e la realtà
  errore = y - y_pred
  # aggiorniamo i pesi e il bias
  bias += errore
  for i in range(len(w)):
    w[i] += lr * errore * x[i]
  return w, bias

# Mettiamo tutto insieme in un esempio

Supponiamo di voler creare un percettrone in grado di imparare il comportamento della porta logica `AND`.

Il dataset di train sara composto dalle coppie di valori booleani $p$ e $q$ mentre la classe associata ad ogni coppia (il valore _target_) corrisponde all'operazione $p \land q$.

|p|q|p $\land$ q|
|:-:|:-:|:---------:|
|0|0|0|
|0|1|0|
|1|0|0|
|1|1|1|

Definiamo il dataset di addestramento

In [25]:
x_train = [[1,0],
           [0,0],
           [1,1],
           [0,0]]
y_train = [1,1,1,0]

Inizializziamo i valori del vettore dei pesi in modo pseudo-random

In [26]:
import random
# inizializzo una casualmente la lista di pesi 
w = [random.uniform(-1,1) for i in range(len(x))]
# inizializza casualmente il bias
bias = random.uniform(-1,1)
w, bias

([-0.42523864157996694, 0.7342500757978592], 0.765425420765975)

Addestriamo il modello per ogni valore dell'input del **train dataset**

In [27]:
def train_once(x_train, y_train, w, bias):
  for i in range(len(x_train)):
    w, bias = train_perceptron(x_train[i], w, bias, y_train[i], f=sign)
  return w, bias

train_once(x_train, y_train, w, bias)

([-0.42523864157996694, 0.7342500757978592], -0.23457457923402503)

Facciamo un **test** del modello sui dati in **train** 

In [28]:
for x in x_train:
  print(x, perceptron(x, w, bias))

[1, 0] 1
[0, 0] 1
[1, 1] 1
[0, 0] 1


Addestriamo per un numero arbitrario di "giri" dell'input (diciamo 10)

In [29]:
epochs = 10
def train_epochs(x_train, y_train, w, bias, epochs):
  for _ in range(epochs):
    print(w, bias)
    w, bias = train_once(x_train, y_train, w, bias)

train_epochs(x_train, y_train, w, bias, epochs)  

[-0.42523864157996694, 0.7342500757978592] 0.765425420765975
[-0.42523864157996694, 0.7342500757978592] -0.23457457923402503
[-0.22523864157996693, 0.7342500757978592] 0.765425420765975
[-0.22523864157996693, 0.7342500757978592] -0.23457457923402503
[-0.025238641579966914, 0.7342500757978592] 0.765425420765975
[-0.025238641579966914, 0.7342500757978592] -0.23457457923402503
[0.1747613584200331, 0.7342500757978592] 0.765425420765975
[0.1747613584200331, 0.7342500757978592] -0.23457457923402503
[0.3747613584200331, 0.7342500757978592] 0.765425420765975
[0.3747613584200331, 0.7342500757978592] -0.23457457923402503


In [30]:
for x in x_train:
  print(x, perceptron(x, w, bias))

[1, 0] 1
[0, 0] 1
[1, 1] 1
[0, 0] 1


# Grazie per la vostra attenzione :-)

Sono a disposizione per chiarimenti e discussioni, anche per eventuali idee di progetto sulle Reti Neurali. 

Mi potete contattare alla mail istituzionali [stefano \[dot\] zingaro \[at\] unibo \[dot\] it](mailto:stefano.zingaro@unibo.it)

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licenza Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />Quest'opera è distribuita con Licenza <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribuzione - Condividi allo stesso modo 4.0 Internazionale</a>. 

# Ringraziamenti

* [Maurizio Gabbrielli](https://www.unibo.it/sitoweb/maurizio.gabbrielli) per aver accolto la mia volontà di efettuare la lezione in questa modalità `python-notebook`.
* [Simone Martini](https://www.cs.unibo.it/~martini/) e [Marco Sbaraglia](https://www.unibo.it/sitoweb/marco.sbaraglia/), i quali hanno enormemente contribuito al miglioramento di questa "lezione". 